In [920]:
import numpy as np
import random

In [921]:
class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        
        self.flights_by_day_dict=self.remove_duplicate(flights_by_day=self.flights_by_day_dict)
        
        self.list_days= [k for k in range(1,self.number_of_areas)]
        
        self.airports_by_area = self.get_airports_by_areas()
        self.area_to_explore=self.which_area_to_explore(airports_by_area=self.airports_by_area)
        self.area_by_airport=self.invert_dict(original_dict=self.airports_by_area)
        
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        self.list_airports=self.get_list_of_airports()
        self.list_areas=list(self.airports_by_area.keys())
        self.areas_connections_by_day=self.possible_flights_from_zone_to_zone_specific_day()
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def flights_from_airport(self,flights_by_day, from_airport, considered_day):
        flights_from_airport = []
        for day, flights in flights_by_day.items():
            if day==considered_day:
                for flight in flights:
                    if flight[0] == from_airport:
                        flights_from_airport.append(flight)
                return flights_from_airport
            else:
                return None

    def invert_dict(self,original_dict):
        inverted_dict = {}
        for key, value_list in original_dict.items():
            for value in value_list:
                if value in inverted_dict:
                    inverted_dict[value].append(key)
                else:
                    inverted_dict[value] = key
        return inverted_dict

    def get_cost(self, day, from_airport, to_airport):
        # Retrieve flights for the specified day and day 0
        flights_day = self.flights_by_day_dict.get(day, [])
        flights_day_0 = self.flights_by_day_dict.get(0, [])
        
        # Find the cost for the specified day
        cost_day = next(
            (flight[2]
            for flight in flights_day
            if flight[0] == from_airport and flight[1] == to_airport),
            float('inf')
        )
        
        # Find the cost for day 0
        cost_day_0 = next(
            (flight[2]
            for flight in flights_day_0
            if flight[0] == from_airport and flight[1] == to_airport),
            float('inf')
        )
        
        # Return the minimum cost if either exists, otherwise inf
        if cost_day == float('inf') and cost_day_0 == float('inf'):
            return float('inf')
        
        return min(cost_day, cost_day_0)

    def possible_flights_from_zone_to_zone_specific_day(self):
        areas_connections_by_day = {}

        for day, flights in self.flights_by_day_dict.items():
            areas_connections_list = []

            for flight in flights:
                connection = f"{self.area_by_airport.get(flight[0])} to {self.area_by_airport.get(flight[1])}"
                if connection not in areas_connections_list:
                    areas_connections_list.append(connection)

            areas_connections_by_day[day] = areas_connections_list

        return areas_connections_by_day

    def get_airports_by_areas(self):
        area_num = int(self.info[0][0])
        return {f"{i}": self.info[2+i * 2] for i in range(0, area_num)}
    
    def get_list_of_airports(self):
        unique_airports = set()

        # Iterate through each sublist and add elements to the set
        for sublist in self.airports_by_area.values():
            for airport in sublist:
                unique_airports.add(airport)
        
        return list(unique_airports)
                    
    def associated_area_to_airport(self,airport):
        return next(
            (
                area
                for area, airports in self.airports_by_area.items()
                if airport in airports
            ),
            "Airport not found",
        ) 
    
    def remove_duplicate(self,flights_by_day):
        for day, flights in flights_by_day.items():
            unique_flights = {}
            for flight in flights:
                flight_key = (flight[0], flight[1])
                if flight_key not in unique_flights:
                    unique_flights[flight_key] = flight
                else:
                    if flight[2] < unique_flights[flight_key][2]:
                        #print(flight[2],unique_flights[flight_key][2])
                        unique_flights[flight_key] = flight
                flights_by_day[day] = list(unique_flights.values())
        return flights_by_day
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                flights_from_airport.append([flight[1], flight[2]])

        return flights_from_airport
    
    def which_area_to_explore(self,airports_by_area):
        return list({key: len(value) for key, value in airports_by_area.items() if len(value) > 1})

In [922]:
Data_Preprocessing=data_preprocessing(instance_path=instance)

In [923]:
class heuristic_operators:
    def __init__(self):
        pass

    @staticmethod
    def swap(arr, a, b):
        arr[a],arr[b] = arr[b], arr[a]

    @staticmethod
    def reverse(arr, a, b):
        if a > b:
            a, b = b, a
        arr[a:b+1] = arr[a:b+1][::-1]

    @staticmethod
    def insert(arr, a, b):
        temp = arr[a]
        del arr[a]
        arr.insert(b, temp)

    @staticmethod
    def swap_k(arr, a, areas_n):
        s = np.random.randint(1, areas_n, size=a * 2)
        for i in range((a // 2) + 2):
            temp = arr[s[i]]
            arr[s[i]] = arr[s[i + 1]]
            arr[s[i + 1]] = temp
            
    @staticmethod
    def swap_target(arr, from_index, to_index):
        arr_copy=arr.copy()
        if from_index >= len(arr_copy) or to_index >= len(arr_copy):
            raise IndexError("Target or position is out of range")
        
        arr_copy[from_index], arr_copy[to_index] = arr_copy[to_index], arr_copy[from_index]
        return arr_copy
    
    @staticmethod
    def swap_area(arr, area_one, area_two):
        arr_copy=arr.copy()
        # Find the indices of area_one and area_two
        index_one = arr_copy.index(area_one)
        index_two = arr_copy.index(area_two)
        
        # Swap the values at these indices
        arr_copy[index_one], arr_copy[index_two] = arr_copy[index_two], arr_copy[index_one]
        
        return arr_copy

In [931]:
class heuristics(data_preprocessing):
    def __init__(self, instance_path):
        
        self.instance_path=instance_path
        super().__init__(instance_path=instance_path)
        
        self.feasible_area_solutions=self.create_feasible_area_solutions(number_iterations=10000)
        self.feasible_airport_solutions=self.find_airports_solutions_from_area_solution(number_iterations=100,area_solutions=self.feasible_area_solutions)
    
    def check_feasibility_area(self, area_solution):

        if area_solution[0]!=area_solution[-1]:
            return False
        
        # Ensure each area is visited exactly once, except the starting/ending area
        area_counts = {area: area_solution.count(area) for area in set(area_solution)}
        
        # Check the counts
        if area_counts[self.starting_area] != 2:
            return False
        for area in self.list_areas:
            if area != self.starting_area and area_counts.get(area, 0) != 1:
                return False
            
        area_solution_feasibility=[]
        for i in range(len(area_solution)-1):
            day = i + min(self.list_days)

            connection = f"{area_solution[i]} to {area_solution[i + 1]}"
            
            if (connection in self.areas_connections_by_day.get(day, [])) or (connection in self.areas_connections_by_day.get(0, [])):
                area_solution_feasibility.append(True)
            else:
                area_solution_feasibility.append(False)

        #print(area_solution_feasibility)
        if False in area_solution_feasibility:
            return False
        
        return True
        
    def check_feasibility_airports(self,airport_solution):
        airport_s=airport_solution.copy()
        airport_solution_feasibility=[]

        # Iterate over the range of the length of the airport solution minus 1
        for k in range(len(airport_s)-1):
            cost_day=self.get_cost(day=k+1,from_airport=airport_solution[k],to_airport=airport_solution[k+1])
            airport_solution_feasibility.append(cost_day)
            
        for k in range(len(airport_solution_feasibility)):
            if airport_solution_feasibility[k]==False:
                return False, airport_solution_feasibility
        return True, sum(airport_solution_feasibility)

    def create_feasible_area_solutions(self,number_iterations):
        area_initial_solution=self.list_areas.copy()
        
        for k in range(len(area_initial_solution)):
            if area_initial_solution[k]==self.starting_area:
                area_initial_solution=heuristic_operators.swap_target(arr=area_initial_solution,
                                                                    from_index=k,
                                                                    to_index=0)
        area_initial_solution.append(self.starting_area)
        
        
        list_solutions=[]
        
        if self.check_feasibility_area(area_initial_solution):
            list_solutions.append(area_initial_solution)
            
        for _ in range(number_iterations):
            x=np.random.randint(1,len(area_initial_solution))
            y=np.random.randint(1,len(area_initial_solution))
            potential_solution=area_initial_solution.copy()
            potential_solution=heuristic_operators.swap_target(potential_solution,from_index=x,to_index=y)
            if self.check_feasibility_area(area_solution=potential_solution) and potential_solution not in list_solutions:
                list_solutions.append(potential_solution)
        return list_solutions
    
    def find_airports_solutions_from_area_solution(self,number_iterations, area_solutions):
        master_solution = {}

        for area_solution in area_solutions:
            airport_solution_d = {}

            airport_solution = area_solution.copy()
            airport_solution[0] = self.starting_airport
            for i in range(1,len(area_solution)):
                airport_solution[i] = self.airports_by_area.get(area_solution[i])[0]
                
            for _ in range(number_iterations):
                potential_airport_solution_x=airport_solution.copy()
                potential_airport_solution_y=airport_solution.copy()
                potential_airport_solution_xy=airport_solution.copy()
                potential_airport_solution_yx=airport_solution.copy()
                
                x=random.randint(1,len(airport_solution)-1)
                y=random.randint(1,len(airport_solution)-1)

                potential_airport_solution_x[x]=random.choice(self.airports_by_area.get(area_solution[x]))
                potential_airport_solution_y[y]=random.choice(self.airports_by_area.get(area_solution[y]))
                potential_airport_solution_xy=heuristic_operators.swap_target(arr=potential_airport_solution_xy,from_index=x,to_index=y)
                potential_airport_solution_yx=heuristic_operators.swap_target(arr=potential_airport_solution_yx,from_index=y,to_index=x)
                
                potential_solutions=[potential_airport_solution_x,potential_airport_solution_y,potential_airport_solution_xy,potential_airport_solution_yx]

                costs=[0,0,0,0]
                
                for k in range(len(potential_solutions)):
                    if self.check_feasibility_airports(potential_solutions[k])[0]==False:
                        costs[k]=np.inf
                    else:
                        costs[k]=self.check_feasibility_airports(airport_solution=potential_solutions[k])[1]
                
                minimum=min(costs)
                solution=tuple(potential_solutions[costs.index(minimum)])
                #print(minimum,solution)
                
                if solution not in airport_solution_d.keys():
                    airport_solution_d[tuple(solution)]=minimum
                    
                sorted_airport_solution = dict(sorted(airport_solution_d.items(), key=lambda item: item[1], reverse=False))
                
        #sorted_airport_solution = dict(sorted(airport_solution_d.items(), key=lambda item: item[1], reverse=False))
            master_solution[tuple(area_solution)] = sorted_airport_solution
        return master_solution

In [939]:
instance="Flight connections dataset/4.in"
Heuristics=heuristics(instance_path=instance)

KeyboardInterrupt: 

In [ ]:
a=Heuristics.feasible_airport_solutions

In [ ]:
keys=list(a.keys())

min_d={}
for k in range(len(keys)):
    min_d[keys[k]]=list(a.get(keys[k]).values())[0]
    
min(list(min_d.values()))

11772